## Install Libraries


In [20]:
%pip install "ray[tune]"
# %pip install torch torchvision torchaudio # Uncomment this to install PyTorch 2.0+ on ilab (required version to run this program)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Libraries


In [21]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict


## Data loaders


In [22]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset


## Configurable neural network


In [23]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)


In [24]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

#channel_1 = 16
#channel_2 = 32
#channel_3 = 64
pool_kernel_size = 2

kernel_size_1 = 3
kernel_size_2 = 5
kernel_size_3 = 7

pad_size_1 = 2
pad_size_2 = 2
pad_size_3 = 3

class Net(nn.Module):
    def __init__(self, channel_1, channel_2, channel_3, fc_count_1):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        self.fc2 = nn.Linear(fc_count_1, fc_count_1)
        self.fc3 = nn.Linear(fc_count_1, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x



## The train function


In [25]:
def train_cifar(config):
    net = Net(
        config["channel_1"], 
        config["channel_2"], 
        config["channel_3"], 
        config["fc_count_1"]
    )  # Will need to modify the parameters depending on what you are experimenting with

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=1e-3)

    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")


## Test set accuracy


In [26]:
def test_best_model(best_result):
    best_trained_model = Net(
        best_result.config["channel_1"], 
        best_result.config["channel_2"], 
        best_result.config["channel_3"], 
        best_result.config["fc_count_1"] 
     )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")


## Configuring the search space


In [27]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter
config = {"channel_1": tune.grid_search([16, 32]), 
          "channel_2": tune.grid_search([32, 64]), 
          "channel_3": tune.grid_search([64, 128]), 
          "fc_count_1": tune.grid_search([256, 512, 1024])}



In [28]:
# Do not modify any code below!
def main(config):
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"gpu": 1},
        ),
        param_space=config,
    )
    results = tuner.fit()

    best_result = results.get_best_result("accuracy", "max")

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    test_best_model(best_result)

    return results.get_dataframe("accuracy", "max")


results_df = main(config)


(train_cifar pid=120319) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_06-59-24/train_cifar_e50ea_00000_0_channel_1=16,channel_2=32,channel_3=64,fc_count_1=256_2023-04-08_06-59-24/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:05<00:00, 29569525.97it/s]


(train_cifar pid=120319) Extracting /root/ray_results/train_cifar_2023-04-08_06-59-24/train_cifar_e50ea_00000_0_channel_1=16,channel_2=32,channel_3=64,fc_count_1=256_2023-04-08_06-59-24/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_06-59-24/train_cifar_e50ea_00000_0_channel_1=16,channel_2=32,channel_3=64,fc_count_1=256_2023-04-08_06-59-24/data
(train_cifar pid=120319) Files already downloaded and verified


Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_e50ea_00000,0.5957,2023-04-08_07-00-28,True,,dc559d4592254b2788bffefcffca1f18,"0_channel_1=16,channel_2=32,channel_3=64,fc_count_1=256",e51e7e6c9f35,10,1.13852,172.28.0.12,120319,True,60.8397,4.6523,60.8397,1680937228,0,,10,e50ea_00000,0.00384831
train_cifar_e50ea_00001,0.6304,2023-04-08_07-01-25,True,,dc559d4592254b2788bffefcffca1f18,"1_channel_1=32,channel_2=32,channel_3=64,fc_count_1=256",e51e7e6c9f35,10,1.04847,172.28.0.12,120319,True,57.5391,4.74419,57.5391,1680937285,0,,10,e50ea_00001,0.00384831
train_cifar_e50ea_00002,0.6129,2023-04-08_07-02-22,True,,dc559d4592254b2788bffefcffca1f18,"2_channel_1=16,channel_2=64,channel_3=64,fc_count_1=256",e51e7e6c9f35,10,1.10462,172.28.0.12,120319,True,56.9936,4.6174,56.9936,1680937342,0,,10,e50ea_00002,0.00384831
train_cifar_e50ea_00003,0.641,2023-04-08_07-03-19,True,,dc559d4592254b2788bffefcffca1f18,"3_channel_1=32,channel_2=64,channel_3=64,fc_count_1=256",e51e7e6c9f35,10,1.02273,172.28.0.12,120319,True,56.988,4.656,56.988,1680937399,0,,10,e50ea_00003,0.00384831
train_cifar_e50ea_00004,0.6121,2023-04-08_07-04-16,True,,dc559d4592254b2788bffefcffca1f18,"4_channel_1=16,channel_2=32,channel_3=128,fc_count_1=256",e51e7e6c9f35,10,1.11323,172.28.0.12,120319,True,56.5805,4.64603,56.5805,1680937456,0,,10,e50ea_00004,0.00384831
train_cifar_e50ea_00005,0.6293,2023-04-08_07-05-15,True,,dc559d4592254b2788bffefcffca1f18,"5_channel_1=32,channel_2=32,channel_3=128,fc_count_1=256",e51e7e6c9f35,10,1.04657,172.28.0.12,120319,True,58.3884,4.67642,58.3884,1680937515,0,,10,e50ea_00005,0.00384831
train_cifar_e50ea_00006,0.6301,2023-04-08_07-06-12,True,,dc559d4592254b2788bffefcffca1f18,"6_channel_1=16,channel_2=64,channel_3=128,fc_count_1=256",e51e7e6c9f35,10,1.05171,172.28.0.12,120319,True,57.9438,4.63474,57.9438,1680937572,0,,10,e50ea_00006,0.00384831
train_cifar_e50ea_00007,0.6569,2023-04-08_07-07-10,True,,dc559d4592254b2788bffefcffca1f18,"7_channel_1=32,channel_2=64,channel_3=128,fc_count_1=256",e51e7e6c9f35,10,0.98973,172.28.0.12,120319,True,57.9198,4.68373,57.9198,1680937630,0,,10,e50ea_00007,0.00384831
train_cifar_e50ea_00008,0.5906,2023-04-08_07-08-07,True,,dc559d4592254b2788bffefcffca1f18,"8_channel_1=16,channel_2=32,channel_3=64,fc_count_1=512",e51e7e6c9f35,10,1.14004,172.28.0.12,120319,True,56.6461,4.69778,56.6461,1680937687,0,,10,e50ea_00008,0.00384831
train_cifar_e50ea_00009,0.6305,2023-04-08_07-09-04,True,,dc559d4592254b2788bffefcffca1f18,"9_channel_1=32,channel_2=32,channel_3=64,fc_count_1=512",e51e7e6c9f35,10,1.06416,172.28.0.12,120319,True,56.7272,4.65857,56.7272,1680937744,0,,10,e50ea_00009,0.00384831


(train_cifar pid=120319) Finished Training
(train_cifar pid=120319) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_06-59-24/train_cifar_e50ea_00001_1_channel_1=32,channel_2=32,channel_3=64,fc_count_1=256_2023-04-08_07-00-28/data/cifar-10-python.tar.gz


...

(train_cifar pid=120319) Extracting /root/ray_results/train_cifar_2023-04-08_06-59-24/train_cifar_e50ea_00022_22_channel_1=16,channel_2=64,channel_3=128,fc_count_1=1024_2023-04-08_07-20-39/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_06-59-24/train_cifar_e50ea_00022_22_channel_1=16,channel_2=64,channel_3=128,fc_count_1=1024_2023-04-08_07-20-39/data
(train_cifar pid=120319) Files already downloaded and verified
(train_cifar pid=120319) Finished Training
(train_cifar pid=120319) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_06-59-24/train_cifar_e50ea_00023_23_channel_1=32,channel_2=64,channel_3=128,fc_count_1=1024_2023-04-08_07-21-36/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:06<00:00, 26651479.80it/s]


(train_cifar pid=120319) Extracting /root/ray_results/train_cifar_2023-04-08_06-59-24/train_cifar_e50ea_00023_23_channel_1=32,channel_2=64,channel_3=128,fc_count_1=1024_2023-04-08_07-21-36/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_06-59-24/train_cifar_e50ea_00023_23_channel_1=32,channel_2=64,channel_3=128,fc_count_1=1024_2023-04-08_07-21-36/data
(train_cifar pid=120319) Files already downloaded and verified
(train_cifar pid=120319) Finished Training


2023-04-08 07:22:34,920	INFO tune.py:798 -- Total run time: 1390.84 seconds (1390.80 seconds for the tuning loop).


Best trial config: {'channel_1': 32, 'channel_2': 64, 'channel_3': 64, 'fc_count_1': 1024}
Best trial final validation loss: 0.9753413572432889
Best trial final validation accuracy: 0.6614
Files already downloaded and verified
Files already downloaded and verified
Best trial test set accuracy: 0.6072


In [29]:
final_df = results_df.drop(
    columns=[
        "should_checkpoint",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)
final_df.sort_values(by=['accuracy'], ascending=False).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/channel_1,config/channel_2,config/channel_3,config/fc_count_1
19,0.975341,0.6614,4.878757,False,59.417617,32,64,64,1024
7,0.989730,0.6569,4.683729,False,57.919765,32,64,128,256
21,1.013896,0.6478,4.904356,False,59.467449,32,32,128,1024
18,1.018645,0.6429,4.798695,False,57.242782,16,64,64,1024
3,1.022734,0.6410,4.656001,False,56.988004,32,64,64,256
17,1.031187,0.6404,4.995471,False,57.296867,32,32,64,1024
15,1.013337,0.6402,4.885772,False,57.224887,32,64,128,512
13,1.046124,0.6364,4.808583,False,57.912948,32,32,128,512
23,1.040542,0.6338,4.877846,False,58.472008,32,64,128,1024
11,1.033950,0.6330,4.760904,False,58.300602,32,64,64,512


In [30]:
final_df.sort_values(by=['accuracy']).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/channel_1,config/channel_2,config/channel_3,config/fc_count_1
8,1.140040,0.5906,4.697781,False,56.646096,16,32,64,512
0,1.138516,0.5957,4.652296,False,60.839730,16,32,64,256
14,1.109301,0.6016,4.882533,False,56.891291,16,64,128,512
16,1.132881,0.6024,4.923333,False,58.026487,16,32,64,1024
20,1.097303,0.6104,4.823255,False,58.260679,16,32,128,1024
12,1.102692,0.6116,4.788044,False,57.963859,16,32,128,512
4,1.113232,0.6121,4.646035,False,56.580479,16,32,128,256
2,1.104625,0.6129,4.617398,False,56.993552,16,64,64,256
10,1.065282,0.6262,4.748732,False,56.389510,16,64,64,512
5,1.046574,0.6293,4.676423,False,58.388433,32,32,128,256
